In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
#!pip install tensorflow
# Set the seed value for the notebook so the results are reproducible
# from numpy.random import seed
# seed(1)

In [4]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
#from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Read the CSV and Perform Basic Data Cleaning

In [5]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [6]:
# Create a random forest classifier
target = df["koi_disposition"]
target_names = ["CONFIRMED", "FALSE POSITIVE", "OTHER"]

data = df.drop("koi_disposition", axis=1)
feature_names = data.columns
data.head()


,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## KOI (Kepler Object of Interest) Columns:
### koi_disposition (y-value) = Exoplanet Archive Disposition (Confirmed = 1)
#### koi_period = Orbital Period (Days)
#### koi_time0bk = Transit Epoch (Center of first detected transit)
#### koi_impact = Impact Parameter (Distance from center of planet/stellar disc)
#### koi_duration = Transit Duration (Hours-Between planet/star first/last contact)
#### koi_prad = Planetary Radius
#### koi_teq = Equilibrium Temperature (Kelvin)
#### koi_model_snr = Transit depth normalized by mean uncertainty

# Select your features (columns)

In [7]:
# Set features. This will also be used as your x values.
# selected_features = df[['koi_disposition', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_prad', 'koi_teq', 'koi_model_snr']]
# selected_features.head()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.5, random_state=1)

In [9]:
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X_train, y_train)
# clf.score(X_test, y_test)

In [10]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8973112128146453

In [11]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True) 

[(0.09749497471480897, 'koi_fpflag_co'),
 (0.08197104407623378, 'koi_fpflag_nt'),
 (0.0637148959252222, 'koi_fpflag_ss'),
 (0.05841114231330306, 'koi_model_snr'),
 (0.05648886950153341, 'koi_prad'),
 (0.03785890805529461, 'koi_prad_err1'),
 (0.037453946512498955, 'koi_fpflag_ec'),
 (0.03702753632956058, 'koi_steff_err1'),
 (0.0361544547210989, 'koi_duration_err2'),
 (0.03325582979914098, 'koi_prad_err2'),
 (0.029254294183742143, 'koi_duration_err1'),
 (0.025090547974171606, 'koi_steff_err2'),
 (0.023365931218129733, 'koi_period'),
 (0.023206674230629422, 'koi_duration'),
 (0.02038860263277005, 'koi_time0bk_err2'),
 (0.020187492527748433, 'koi_insol_err1'),
 (0.019547590093508194, 'koi_time0bk_err1'),
 (0.01929712350623025, 'koi_depth'),
 (0.019265276698821426, 'koi_impact'),
 (0.01709370209660179, 'koi_period_err1'),
 (0.016701874145672382, 'koi_insol'),
 (0.016699482045615735, 'koi_period_err2'),
 (0.01597287477622853, 'koi_teq'),
 (0.01592160605634961, 'koi_time0bk'),
 (0.01543710117

In [12]:
selected_features = df[['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_co', 'koi_fpflag_ss', 'koi_model_snr']]
selected_features.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ss,koi_model_snr
0,CONFIRMED,0,0,0,25.8
1,FALSE POSITIVE,0,0,1,76.3
2,FALSE POSITIVE,0,0,1,505.6
3,CONFIRMED,0,0,0,40.9
4,CONFIRMED,0,0,0,40.2


# Create a Train Test Split

Use `koi_disposition` for the y values

In [13]:
X = selected_features.drop('koi_disposition', axis=1).astype(int)
y = selected_features['koi_disposition']
print(X.shape, y.shape)
X.dtypes


(6991, 4) (6991,)


koi_fpflag_nt    int64
koi_fpflag_co    int64
koi_fpflag_ss    int64
koi_model_snr    int64
dtype: object

In [14]:


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=1, stratify=y)
X_train.head()

,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ss,koi_model_snr
3382,0,1,0,13
4054,0,1,1,195
6361,0,0,1,1752
5005,0,0,0,6
1702,0,0,0,21


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [15]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [16]:
print(y_train_categorical)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


# Train the Model



In [17]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [18]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               500       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 10,903
Trainable params: 10,903
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
110/110 - 1s - loss: 0.6168 - accuracy: 0.6758
Epoch 2/60
110/110 - 0s - loss: 0.4376 - accuracy: 0.7413
Epoch 3/60
110/110 - 0s - loss: 0.4349 - accuracy: 0.7373
Epoch 4/60
110/110 - 0s - loss: 0.4340 - accuracy: 0.7388
Epoch 5/60
110/110 - 0s - loss: 0.4328 - accuracy: 0.7413
Epoch 6/60
110/110 - 0s - loss: 0.4316 - accuracy: 0.7422
Epoch 7/60
110/110 - 0s - loss: 0.4328 - accuracy: 0.7488
Epoch 8/60
110/110 - 0s - loss: 0.4328 - accuracy: 0.7396
Epoch 9/60
110/110 - 0s - loss: 0.4315 - accuracy: 0.7439
Epoch 10/60
110/110 - 0s - loss: 0.4308 - accuracy: 0.7488
Epoch 11/60
110/110 - 0s - loss: 0.4302 - accuracy: 0.7525
Epoch 12/60
110/110 - 0s - loss: 0.4310 - accuracy: 0.7614
Epoch 13/60
110/110 - 0s - loss: 0.4300 - accuracy: 0.7594
Epoch 14/60
110/110 - 0s - loss: 0.4302 - accuracy: 0.7605
Epoch 15/60
110/110 - 0s - loss: 0.4302 - accuracy: 0.7568
Epoch 16/60
110/110 - 0s - loss: 0.4294 - accuracy: 0.7542
Epoch 17/60
110/110 - 0s - loss: 0.4288 - accuracy: 0.7574
Epoch 

In [20]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

110/110 - 0s - loss: 0.4063 - accuracy: 0.8372
Normal Neural Network - Loss: 0.4062722623348236, Accuracy: 0.8372425436973572


In [21]:
# deep_model = Sequential()
# deep_model.add(Dense(units=6, activation='relu', input_dim=4))
# deep_model.add(Dense(units=6, activation='relu'))
# deep_model.add(Dense(units=3, activation='softmax'))
# deep_model.summary()

In [22]:
# deep_model.compile(optimizer='adam',
#                    loss='categorical_crossentropy',
#                    metrics=['accuracy'])

# deep_model.fit(
#     X_train_scaled,
#     y_train_categorical,
#     epochs=100,
#     shuffle=True,
#     verbose=2
# )

In [23]:
# model_loss, model_accuracy = deep_model.evaluate(
#     X_test_scaled, y_test_categorical, verbose=2)
# print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [24]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [25]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['CONFIRMED' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'FALSE POSITIVE']
Actual Labels: ['CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE']


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [26]:
# Create the GridSearchCV model
model = SVC(kernel='linear')
model

SVC(kernel='linear')

In [27]:
SVC(kernel='linear')

SVC(kernel='linear')

In [28]:
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [29]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.737, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.747, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.748, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.741, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.740, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.737, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.747, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.748, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] .

[CV] ................... C=10, gamma=0.005, score=0.740, total=   0.0s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.737, total=   0.0s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.747, total=   0.0s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.748, total=   0.0s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.741, total=   0.0s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.740, total=   0.0s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .................. C=50, gamma=0.0005, score=0.737, total=   0.0s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    3.0s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [30]:
GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             verbose=2)

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             verbose=2)

In [31]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'gamma': 0.0001}
0.742489270386266


In [32]:
 # Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)
print(predictions)

['CONFIRMED' 'CONFIRMED' 'FALSE POSITIVE' ... 'CONFIRMED' 'FALSE POSITIVE'
 'CONFIRMED']


In [33]:
print('Test Acc: %.3f' % grid.score(X_test_scaled, y_test))

Test Acc: 0.736


In [34]:
# from sklearn import metrics
# from sklearn.metrics import f1_score
# #metrics.f1_score(y_test, predictions, labels=np.unique(predictions))
# metrics.f1_score(y_test, predictions, average='weighted', labels=np.unique(predictions))


In [35]:
print(classification_report(y_test, predictions,
                            target_names=["CONFIRMED", "FALSE POSITIVE", "OTHER"]))
# print(classification_report(y_test, predictions,
#                             target_names=['koi_disposition_CONFIRMED', 'koi_time0bk']))
# print(classification_report(y_test, predictions,
#                             target_names=['koi_disposition_CONFIRMED', 'koi_impact']))
# print(classification_report(y_test, predictions,
#                             target_names=['koi_disposition_CONFIRMED', 'koi_duration']))
# print(classification_report(y_test, predictions,
#                             target_names=['koi_disposition_CONFIRMED', 'koi_prad']))
# print(classification_report(y_test, predictions,
#                             target_names=['koi_disposition_CONFIRMED', 'koi_teq']))
# print(classification_report(y_test, predictions,
#                             target_names=['koi_disposition_CONFIRMED', 'koi_model_snr']))

                precision    recall  f1-score   support

     CONFIRMED       0.00      0.00      0.00       844
FALSE POSITIVE       0.50      0.97      0.66       900
         OTHER       0.97      0.97      0.97      1752

      accuracy                           0.74      3496
     macro avg       0.49      0.65      0.54      3496
  weighted avg       0.62      0.74      0.66      3496



/Users/serenabaker/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Save the Model

In [36]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

# import joblib
# filename = 'serena_baker.sav'
# joblib.dump(model2, filename)